In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data= pd.read_csv("churn_telco.csv")

In [4]:
data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [13]:
# Data Overview
print("Rows",data.shape[0])
print("Columns",data.shape[1])
print("Missing values",data.isna().sum().values.sum())
print("Unique Values",data.nunique())

Rows 7043
Columns 21
Missing values 0
Unique Values customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64


tenure has 73 unique numbers distributed among 7043 customer Id's. 
Converting tenure from monthly to yearly makes more sense. it will help in building a good model as well. 

In [44]:
#replacing empty values in total charges to null
data["TotalCharges"]= data["TotalCharges"].replace(" ",np.nan)

data= data[data["TotalCharges"].notnull()]
data=data.reset_index()[data.columns]

data["TotalCharges"]= data["TotalCharges"].astype(float)

#replacing no internet service on some columns to just no
cols_no_internet = ["OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]

for cols in cols_no_internet:
    data[cols]=data[cols].replace({'No internet service':'No'})
    
#senior citizens replace values
data["SeniorCitizen"]=data["SeniorCitizen"].replace({1:'Yes',0:'No'})

# converting monthly tenure to yearly tenure

def tenure_convert(data):
    if (data['tenure']>0) & (data['tenure']<=12):
        return 'tenure_Year1'
    elif (data['tenure']>12) & (data['tenure']<=24):
        return 'tenure_Year2'
    elif (data['tenure']>24) & (data['tenure'] <=36):
        return 'tenure_Year3'
    elif (data['tenure']>36) & (data['tenure'] <= 48):
        return 'tenure_Year4'
    elif (data['tenure']>48) & (data['tenure'] <= 60):
        return 'tenure_Year5'
    elif (data['tenure']>60) & (data['tenure'] <= 72):
        return 'tenure_Year6'
    
data["tenure_year"]= data.apply(lambda data:tenure_convert(data),axis=1)
    

In [32]:
data=data.reset_index()[data.columns]

In [39]:
data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [45]:
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_year
0,7590-VHVEG,Female,No,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,tenure_Year1
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.50,No,tenure_Year3
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,tenure_Year1
3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,tenure_Year4
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,tenure_Year1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,6840-RESVB,Male,No,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No,tenure_Year2
7028,2234-XADUH,Female,No,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No,tenure_Year6
7029,4801-JZAZL,Female,No,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No,tenure_Year1
7030,8361-LTMKD,Male,Yes,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes,tenure_Year1


In [ ]:
#Scaling and Encoding